# Chapter 3: Fibonacci Example–Part 2
Now, we will generate multiple witnesses to test the soundness of our circuit constraints. Note that we only intend to accept the following set of values for signals "a", "b", and "c". "Soundness" in this context refers to faulty witness successfully verified against the constraints (false positives), so any set of witness assignments that is different from the table below but still passes the constraints incurs a "soundness" error.
| a | b | c |
| - | - | - |
| 1 | 1 | 2 |
| 1 | 2 | 3 |
| 2 | 3 | 5 |
| 3 | 5 | 8 |
| ... | |   |

## Setup
We setup the same circuit and witness in Part 1 which were successfully verified:

In [ ]:
from __future__ import annotations
from typing import Tuple

from dsl import Circuit, StepType
from cb import eq
from util import F

class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")

        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        
    def trace(self, args):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 10):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

fibo = Fibonacci()
fibo_witness = fibo.gen_witness(None)